In [1]:
import pandas as pd
import numpy as np
import json
from scipy import sparse as sp
from tqdm.notebook import tqdm
from collections import defaultdict

In [2]:
import implicit

In [3]:
import sys
sys.path.append('../')

from src.utils import get_shard_path
from src.utils import ProductEncoder, make_coo_row
from src.metrics import normalized_average_precision

In [4]:
product_encoder = ProductEncoder('data/raw/products.csv')

In [5]:
valid_data = [json.loads(l) for l in open(get_shard_path(7))][:3000]

In [6]:
rows = []
for shard_id in range(4):
    for js in tqdm(json.loads(l) for l in open(get_shard_path(shard_id))):
        rows.append(make_coo_row(js["transaction_history"], product_encoder))

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [7]:
X_sparse = sp.vstack(rows).tocsr()

In [8]:
X_sparse.shape

(183938, 43038)

# ALS

In [9]:
model = implicit.als.AlternatingLeastSquares(factors=16, regularization=0.0, iterations=8)
model.fit(X_sparse)

/home/sap/src/personal/recommendation-systems/retailhero-recomender-baseline/.venv/lib/python3.12/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 32 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/8 [00:00<?, ?it/s]

In [10]:
max_index = max(product_encoder.product_pid.keys())
print(f"Maximum index in ProductEncoder: {max_index}")

Maximum index in ProductEncoder: 43037


In [11]:
m_ap = []
for js in tqdm(valid_data):
    row_sparse = make_coo_row(js["transaction_history"], product_encoder).tocsr()
    raw_recs = model.recommend(0, row_sparse, N=30, filter_already_liked_items=False, recalculate_user=True)
    recommended_items = product_encoder.toPid(raw_recs[0])
    gt_items = js["target"][0]["product_ids"]
    m_ap.append(normalized_average_precision(gt_items, recommended_items, k=30))
print(np.mean(m_ap))

  0%|          | 0/3000 [00:00<?, ?it/s]

0.0733627756563659


In [12]:
raw_recs

(array([31933, 37296, 10510,  8833, 34153, 40445, 12454, 41355, 14501,
        26079, 20857,  4740, 36370, 33094, 12790, 18300,  8809, 12982,
         3698,  7093, 18949,  7049, 41289, 13721, 38754, 30388, 41615,
         7729, 16845, 42234], dtype=int32),
 array([0.50665104, 0.42714727, 0.41047382, 0.401904  , 0.32832253,
        0.32735538, 0.32180572, 0.3149966 , 0.31002355, 0.228138  ,
        0.22199476, 0.22142145, 0.21749288, 0.21690558, 0.20663902,
        0.2058504 , 0.20087549, 0.19946828, 0.18473189, 0.17346415,
        0.17304045, 0.16482276, 0.1542864 , 0.14809377, 0.14717445,
        0.14593145, 0.14526415, 0.14440086, 0.13701133, 0.13634972],
       dtype=float32))

In [13]:
# ???